In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting .\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting .\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting .\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting .\t10k-labels-idx1-ubyte.gz


In [2]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

## Function to Generate Convolutional Layers

![Convolution Example](./figs/convolution-schematic.gif)

In [3]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

## Define a Function to Generate Max-Pooling Layers

![Max-Pooling Example](./figs/maxpool.jpeg)

In [4]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

## Define the Network

In [5]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 50883.2500 Validation Accuracy: 0.156250
Epoch  1, Batch   2 -Loss: 36976.0391 Validation Accuracy: 0.175781
Epoch  1, Batch   3 -Loss: 34733.1094 Validation Accuracy: 0.160156
Epoch  1, Batch   4 -Loss: 30422.4160 Validation Accuracy: 0.179688
Epoch  1, Batch   5 -Loss: 27062.7969 Validation Accuracy: 0.203125
Epoch  1, Batch   6 -Loss: 26856.1289 Validation Accuracy: 0.207031
Epoch  1, Batch   7 -Loss: 26343.0938 Validation Accuracy: 0.226562
Epoch  1, Batch   8 -Loss: 25244.4688 Validation Accuracy: 0.246094
Epoch  1, Batch   9 -Loss: 25531.0352 Validation Accuracy: 0.210938
Epoch  1, Batch  10 -Loss: 22617.3086 Validation Accuracy: 0.218750
Epoch  1, Batch  11 -Loss: 20246.9531 Validation Accuracy: 0.218750
Epoch  1, Batch  12 -Loss: 19524.8770 Validation Accuracy: 0.246094
Epoch  1, Batch  13 -Loss: 23385.9023 Validation Accuracy: 0.242188
Epoch  1, Batch  14 -Loss: 19997.1309 Validation Accuracy: 0.238281
Epoch  1, Batch  15 -Loss: 19200.3438 Validation

Epoch  1, Batch 122 -Loss:  3390.2444 Validation Accuracy: 0.617188
Epoch  1, Batch 123 -Loss:  4714.2402 Validation Accuracy: 0.617188
Epoch  1, Batch 124 -Loss:  2280.0337 Validation Accuracy: 0.621094
Epoch  1, Batch 125 -Loss:  4136.2349 Validation Accuracy: 0.621094
Epoch  1, Batch 126 -Loss:  4123.0352 Validation Accuracy: 0.636719
Epoch  1, Batch 127 -Loss:  3849.8574 Validation Accuracy: 0.628906
Epoch  1, Batch 128 -Loss:  3599.0991 Validation Accuracy: 0.625000
Epoch  1, Batch 129 -Loss:  4030.1484 Validation Accuracy: 0.628906
Epoch  1, Batch 130 -Loss:  4802.4585 Validation Accuracy: 0.628906
Epoch  1, Batch 131 -Loss:  3248.5576 Validation Accuracy: 0.632812
Epoch  1, Batch 132 -Loss:  2967.0938 Validation Accuracy: 0.648438
Epoch  1, Batch 133 -Loss:  4298.8008 Validation Accuracy: 0.648438
Epoch  1, Batch 134 -Loss:  3766.0654 Validation Accuracy: 0.652344
Epoch  1, Batch 135 -Loss:  4196.4062 Validation Accuracy: 0.652344
Epoch  1, Batch 136 -Loss:  4101.4229 Validation

Epoch  1, Batch 243 -Loss:  3017.9902 Validation Accuracy: 0.753906
Epoch  1, Batch 244 -Loss:  3066.6592 Validation Accuracy: 0.742188
Epoch  1, Batch 245 -Loss:  1464.3868 Validation Accuracy: 0.734375
Epoch  1, Batch 246 -Loss:  2910.0034 Validation Accuracy: 0.734375
Epoch  1, Batch 247 -Loss:  3379.9622 Validation Accuracy: 0.738281
Epoch  1, Batch 248 -Loss:  3186.4185 Validation Accuracy: 0.738281
Epoch  1, Batch 249 -Loss:  2002.2847 Validation Accuracy: 0.742188
Epoch  1, Batch 250 -Loss:  2350.1108 Validation Accuracy: 0.730469
Epoch  1, Batch 251 -Loss:  3283.8203 Validation Accuracy: 0.738281
Epoch  1, Batch 252 -Loss:  2319.5225 Validation Accuracy: 0.742188
Epoch  1, Batch 253 -Loss:  1386.0620 Validation Accuracy: 0.738281
Epoch  1, Batch 254 -Loss:  2648.2979 Validation Accuracy: 0.746094
Epoch  1, Batch 255 -Loss:  1574.5664 Validation Accuracy: 0.746094
Epoch  1, Batch 256 -Loss:  1811.1106 Validation Accuracy: 0.750000
Epoch  1, Batch 257 -Loss:  1772.7283 Validation

Epoch  1, Batch 364 -Loss:  1398.9912 Validation Accuracy: 0.781250
Epoch  1, Batch 365 -Loss:  1152.7950 Validation Accuracy: 0.785156
Epoch  1, Batch 366 -Loss:  2041.5189 Validation Accuracy: 0.785156
Epoch  1, Batch 367 -Loss:  1186.7493 Validation Accuracy: 0.789062
Epoch  1, Batch 368 -Loss:  1140.4250 Validation Accuracy: 0.789062
Epoch  1, Batch 369 -Loss:  1478.8391 Validation Accuracy: 0.792969
Epoch  1, Batch 370 -Loss:  1652.9929 Validation Accuracy: 0.800781
Epoch  1, Batch 371 -Loss:  2102.6079 Validation Accuracy: 0.804688
Epoch  1, Batch 372 -Loss:  1718.8746 Validation Accuracy: 0.789062
Epoch  1, Batch 373 -Loss:  2427.6353 Validation Accuracy: 0.792969
Epoch  1, Batch 374 -Loss:  1432.3134 Validation Accuracy: 0.792969
Epoch  1, Batch 375 -Loss:  1394.5112 Validation Accuracy: 0.796875
Epoch  1, Batch 376 -Loss:  1889.9238 Validation Accuracy: 0.796875
Epoch  1, Batch 377 -Loss:  1551.9862 Validation Accuracy: 0.785156
Epoch  1, Batch 378 -Loss:  1626.5148 Validation

Epoch  2, Batch  56 -Loss:   965.9545 Validation Accuracy: 0.808594
Epoch  2, Batch  57 -Loss:   957.1573 Validation Accuracy: 0.808594
Epoch  2, Batch  58 -Loss:  1623.9313 Validation Accuracy: 0.808594
Epoch  2, Batch  59 -Loss:   923.7009 Validation Accuracy: 0.812500
Epoch  2, Batch  60 -Loss:  1319.8280 Validation Accuracy: 0.808594
Epoch  2, Batch  61 -Loss:  1577.4768 Validation Accuracy: 0.808594
Epoch  2, Batch  62 -Loss:  1535.1516 Validation Accuracy: 0.808594
Epoch  2, Batch  63 -Loss:   938.6468 Validation Accuracy: 0.812500
Epoch  2, Batch  64 -Loss:  1179.2952 Validation Accuracy: 0.804688
Epoch  2, Batch  65 -Loss:   737.4847 Validation Accuracy: 0.796875
Epoch  2, Batch  66 -Loss:  2111.5486 Validation Accuracy: 0.789062
Epoch  2, Batch  67 -Loss:  1280.6147 Validation Accuracy: 0.792969
Epoch  2, Batch  68 -Loss:  1752.0178 Validation Accuracy: 0.789062
Epoch  2, Batch  69 -Loss:  1277.7778 Validation Accuracy: 0.785156
Epoch  2, Batch  70 -Loss:  1416.1292 Validation

...

Epoch  7, Batch 210 -Loss:   421.5302 Validation Accuracy: 0.851562
Epoch  7, Batch 211 -Loss:   339.1762 Validation Accuracy: 0.855469
Epoch  7, Batch 212 -Loss:   286.7408 Validation Accuracy: 0.855469
Epoch  7, Batch 213 -Loss:   275.6383 Validation Accuracy: 0.855469
Epoch  7, Batch 214 -Loss:   308.3203 Validation Accuracy: 0.855469
Epoch  7, Batch 215 -Loss:   258.5060 Validation Accuracy: 0.855469
Epoch  7, Batch 216 -Loss:   222.8492 Validation Accuracy: 0.855469
Epoch  7, Batch 217 -Loss:   400.1006 Validation Accuracy: 0.855469
Epoch  7, Batch 218 -Loss:   147.4701 Validation Accuracy: 0.859375
Epoch  7, Batch 219 -Loss:   343.2939 Validation Accuracy: 0.859375
Epoch  7, Batch 220 -Loss:   373.1576 Validation Accuracy: 0.855469
Epoch  7, Batch 221 -Loss:   416.2977 Validation Accuracy: 0.863281
Epoch  7, Batch 222 -Loss:   261.5602 Validation Accuracy: 0.855469
Epoch  7, Batch 223 -Loss:   215.9471 Validation Accuracy: 0.859375
Epoch  7, Batch 224 -Loss:   210.7435 Validation

Epoch  7, Batch 331 -Loss:   356.6240 Validation Accuracy: 0.867188
Epoch  7, Batch 332 -Loss:   229.2135 Validation Accuracy: 0.863281
Epoch  7, Batch 333 -Loss:   236.1902 Validation Accuracy: 0.863281
Epoch  7, Batch 334 -Loss:   294.6293 Validation Accuracy: 0.859375
Epoch  7, Batch 335 -Loss:   254.9004 Validation Accuracy: 0.859375
Epoch  7, Batch 336 -Loss:   251.6364 Validation Accuracy: 0.863281
Epoch  7, Batch 337 -Loss:   380.2496 Validation Accuracy: 0.859375
Epoch  7, Batch 338 -Loss:   236.4138 Validation Accuracy: 0.863281
Epoch  7, Batch 339 -Loss:   300.5294 Validation Accuracy: 0.859375
Epoch  7, Batch 340 -Loss:   287.7958 Validation Accuracy: 0.863281
Epoch  7, Batch 341 -Loss:   316.3172 Validation Accuracy: 0.855469
Epoch  7, Batch 342 -Loss:   360.4490 Validation Accuracy: 0.855469
Epoch  7, Batch 343 -Loss:   229.3177 Validation Accuracy: 0.863281
Epoch  7, Batch 344 -Loss:   243.0050 Validation Accuracy: 0.855469
Epoch  7, Batch 345 -Loss:   288.5444 Validation

Epoch  8, Batch  23 -Loss:   221.6196 Validation Accuracy: 0.871094
Epoch  8, Batch  24 -Loss:   251.9170 Validation Accuracy: 0.863281
Epoch  8, Batch  25 -Loss:   309.9051 Validation Accuracy: 0.867188
Epoch  8, Batch  26 -Loss:   293.0345 Validation Accuracy: 0.867188
Epoch  8, Batch  27 -Loss:   304.7487 Validation Accuracy: 0.867188
Epoch  8, Batch  28 -Loss:   383.3401 Validation Accuracy: 0.863281
Epoch  8, Batch  29 -Loss:   402.8343 Validation Accuracy: 0.859375
Epoch  8, Batch  30 -Loss:   306.0975 Validation Accuracy: 0.855469
Epoch  8, Batch  31 -Loss:   193.6367 Validation Accuracy: 0.855469
Epoch  8, Batch  32 -Loss:   217.4155 Validation Accuracy: 0.859375
Epoch  8, Batch  33 -Loss:   286.8380 Validation Accuracy: 0.851562
Epoch  8, Batch  34 -Loss:   388.9167 Validation Accuracy: 0.851562
Epoch  8, Batch  35 -Loss:   301.7529 Validation Accuracy: 0.851562
Epoch  8, Batch  36 -Loss:   468.1419 Validation Accuracy: 0.847656
Epoch  8, Batch  37 -Loss:   283.6898 Validation

Epoch  8, Batch 144 -Loss:   365.2997 Validation Accuracy: 0.867188
Epoch  8, Batch 145 -Loss:   356.3869 Validation Accuracy: 0.863281
Epoch  8, Batch 146 -Loss:   418.1240 Validation Accuracy: 0.871094
Epoch  8, Batch 147 -Loss:   364.6065 Validation Accuracy: 0.871094
Epoch  8, Batch 148 -Loss:   323.9066 Validation Accuracy: 0.867188
Epoch  8, Batch 149 -Loss:   325.7520 Validation Accuracy: 0.863281
Epoch  8, Batch 150 -Loss:   253.8893 Validation Accuracy: 0.863281
Epoch  8, Batch 151 -Loss:   305.7520 Validation Accuracy: 0.863281
Epoch  8, Batch 152 -Loss:   349.4308 Validation Accuracy: 0.863281
Epoch  8, Batch 153 -Loss:   168.9005 Validation Accuracy: 0.863281
Epoch  8, Batch 154 -Loss:   210.4308 Validation Accuracy: 0.863281
Epoch  8, Batch 155 -Loss:   211.8322 Validation Accuracy: 0.867188
Epoch  8, Batch 156 -Loss:   363.4644 Validation Accuracy: 0.863281
Epoch  8, Batch 157 -Loss:   356.9551 Validation Accuracy: 0.871094
Epoch  8, Batch 158 -Loss:   391.3121 Validation

Epoch  8, Batch 265 -Loss:   267.8529 Validation Accuracy: 0.859375
Epoch  8, Batch 266 -Loss:   318.2330 Validation Accuracy: 0.863281
Epoch  8, Batch 267 -Loss:   382.4225 Validation Accuracy: 0.863281
Epoch  8, Batch 268 -Loss:   346.3394 Validation Accuracy: 0.863281
Epoch  8, Batch 269 -Loss:   305.0205 Validation Accuracy: 0.863281
Epoch  8, Batch 270 -Loss:   380.9338 Validation Accuracy: 0.859375
Epoch  8, Batch 271 -Loss:   191.6861 Validation Accuracy: 0.859375
Epoch  8, Batch 272 -Loss:   292.0983 Validation Accuracy: 0.863281
Epoch  8, Batch 273 -Loss:   260.6121 Validation Accuracy: 0.867188
Epoch  8, Batch 274 -Loss:   382.7902 Validation Accuracy: 0.871094
Epoch  8, Batch 275 -Loss:   301.4160 Validation Accuracy: 0.867188
Epoch  8, Batch 276 -Loss:   119.7436 Validation Accuracy: 0.867188
Epoch  8, Batch 277 -Loss:   253.8797 Validation Accuracy: 0.859375
Epoch  8, Batch 278 -Loss:   197.4260 Validation Accuracy: 0.859375
Epoch  8, Batch 279 -Loss:   277.0276 Validation